In [ ]:
from src.numpynn import layers, activations, networks, utils, optimizers, losses, metrics, paddings, preprocessing
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
data = unpickle('data/cifar10/test_batch')
features = data[b'data']
labels = np.array(data[b'labels'])

print(f'{features.shape=}')
print(f'{labels.shape=}')

In [ ]:
x_reshaped = features.reshape((10000, 3, 32, -1))
x_reshaped = np.moveaxis(x_reshaped, 1, -1)
x_reshaped.shape

In [ ]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

i = 5
plt.imshow(x_reshaped[i])
print(label_names[labels[i]])

In [ ]:
X = x_reshaped / 255

In [ ]:
Y = preprocessing.one_hot_encode(labels, 10)
Y[:10]

In [ ]:
x_norm = X / 255

In [ ]:
ratio_val = 0.1
ratio_test = 0.1

shuffle_index = np.arange(len(x_norm))
np.random.shuffle(shuffle_index)
x_shuffled = x_norm[shuffle_index]
y_shuffled = Y[shuffle_index]

n1 = int(len(x_norm) * (1 - ratio_val - ratio_test))
n2 = int(len(x_norm) * (1 - ratio_val))

x_train = x_shuffled[:n1]
y_train = y_shuffled[:n1]
x_val = x_shuffled[n1:n2]
y_val = y_shuffled[n1:n2]
x_test = x_shuffled[n2:]
y_test = y_shuffled[n2:]

In [ ]:
print(f'{x_train.shape=}')
print(f'{y_train.shape=}')
print(f'{x_val.shape=}')
print(f'{y_val.shape=}')
print(f'{x_test.shape=}')
print(f'{y_test.shape=}')

In [ ]:
model = networks.FeedForward(input_shape=(32,32,3), layers=[
    layers.Convolution(nr_kernels=8, kernel_size=(3, 3), activation=activations.Tanh(), padding=paddings.Same),
    layers.Convolution(nr_kernels=8, kernel_size=(3, 3), activation=activations.Tanh(), padding=paddings.Same),
    layers.MaxPooling(pooling_window=(2, 2)),
    layers.Convolution(nr_kernels=16, kernel_size=(3, 3), activation=activations.Tanh(), padding=paddings.Same),
    layers.Convolution(nr_kernels=16, kernel_size=(3, 3), activation=activations.Tanh(), padding=paddings.Same),
    layers.MaxPooling(pooling_window=(2, 2)),
    layers.Flatten(),
    layers.Linear(nr_neurons=64, activation=activations.Tanh()),
    layers.Linear(nr_neurons=10, activation=activations.Softmax())
])

In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss_function=losses.Crossentropy(),
    metric=metrics.Accuracy
)

In [ ]:
model.summary()

In [ ]:
model.train(x_train, y_train, epochs=100, batch_size=128, val_data=(x_val, y_val))

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
i = random.randint(0, len(X))
image = X[i]
p, _ = model(np.expand_dims(image, 0))
plt.imshow(image, cmap='gray')

In [ ]:
plt.bar(np.arange(0, 10), p.reshape(10,))

# 0 airplane
# 1 automobile
# 2 bird
# 3 cat
# 4 deer
# 5 dog
# 6 frog
# 7 horse
# 8 ship
# 9 truck